In [ ]:
import pandas as pd
import arcpy
import ipywidgets as widgets
from IPython.core.display import display, HTML

In [ ]:
gdb_path = r"C:\jupyter_lab_is_neat\UPDM_AssetPackage.gdb" # the path to the database that holds the tables I want to look at
field_props_to_display = ["name", "aliasName", "type", "domain", "length"] # the field properties I'm currently interested in seeing

In [ ]:
arcpy.env.workspace = gdb_path # set our workspace so arcpy.ListTables and arcpy.ListFields knows where to find the tables

table_list = arcpy.ListTables() # get a list of tables in the gdb
table_field_dict = {} # make an empty dictionary to store the lists of fields from the table
for table in table_list:
    # for each table in the list of tables, get the fields of the table and add them to the dictionary
    fields = arcpy.ListFields(table)
    table_field_dict[table] = fields

domain_list = arcpy.da.ListDomains(gdb_path) # get a list of the domains in the databse
domain_dict = {} # make an empty dictionary to store the domain information
for domain in domain_list:
    # for each domain in the list, add the domain to the dictionary with the key of the dictionary being the domain name
    domain_dict[domain.name] = domain

In [ ]:
display(HTML("<style>.jp-Notebook.jp-mod-scrollPastEnd::after { display: none; }</style>"))

In [ ]:
table_select_w = widgets.Select(options=table_field_dict.keys()) 
# the table select widget will be used to select the table we want to look at, 
#  we'll initialize it with the list of tables from our database

field_select_w = widgets.Select(options=[]) 
# the field select widget will be used to select the field we want to look at
#  we arent going to initialize it with any options because we need to know what table to display the fields from first

def on_table_field_interact(selected_table, selected_field): # this function will be used by the ipywidgets interactive function below
    if selected_table is not None:
        # if the selected table isn't none, set the options in the field select widget to the fields from this table
        selected_table_fields = table_field_dict[selected_table]
        selected_table_field_names = [f.name for f in selected_table_fields]
        field_select_w.options = selected_table_field_names
    else:
        # if no table is selected, populate the field select widget with no options
        field_select_w.options = []
        
    if selected_field is not None:
        # if there is a field selected in the widget...
        for field in selected_table_fields:
            # loop over all the fields from the selected table
            if field.name == selected_field:
                # if the field name matches the field that's selected make a dictionary of the properties we're interested in
                field_prop_dict = {}
                for prop in field_props_to_display:
                    field_prop_dict[prop] = [getattr(field, prop)]
                # take the dictionary and turn it into a pandas dataframe (simply because they render in jupyterlab really pretty)
                #  yes this is very trivial use of a dataframe, but it works so well!
                field_prop_df = pd.DataFrame.from_dict(field_prop_dict, orient='index', columns=["Value"])
                # display the dataframe in the output of the interactive widget
                display(field_prop_df)
                
                domain_name = field.domain
                if domain_name != "":
                    # if the field has a domain get the domain from the dictionary
                    domain = domain_dict[domain_name]
                    if domain.domainType =="CodedValue":
                        # if the domain is a coded value domain, get the coded values from the domain, and add them to a different dataframe
                        coded_values_df = pd.DataFrame.from_dict(domain.codedValues, orient='index', columns=["Description"])
                        # set a name on the index so it looks pretty when we display it
                        coded_values_df.index.name = 'Code'
                        # boom, display that dataframe too!
                        display(coded_values_df)

# the widgets interactive function takes a function, then the arguments for that function are then set from the values of other widgets and returns a VBox of the input widgets and a new output widget
#  so in our case, we'll give it our on_table_field_interact function
#  then since that function takes a selected_table and selected_field argument, we'll pass the corresponding table and field widgets to the interactive function 
interactive_display_w = widgets.interactive(on_table_field_interact, selected_table=table_select_w, selected_field=field_select_w)

# since the interactive function returns a VBox, and I really want this to be aligned horizontally, not vertically, 
#  we'll get the last child of the VBox wich is the output widget where the field information will be displayed
interactive_display_output_w = interactive_display_w.children[-1]

# when you pass widgets to the interactive function, it sets a desctiption on them of the argument name
#  i would rather the description just be "Table" and "Field" instead of "selected_table" and "selected_field"
#  so we'll overwrite those properties on the widget and set the height and width to be a bit bigger, so we can see more stuff in them
table_select_w.description = "Table"
table_select_w.layout.width = '400px'
table_select_w.layout.height = '400px'

field_select_w.description = "Field"
field_select_w.layout.width = '400px'
field_select_w.layout.height = '400px'

# lastly we need to make a HBox to make the widgets show up horizontally,
#  because for our use it makes a more appealing interface (in my opinion... and im the person writing it :) )
#  and then we need to display the HBox
selectors_and_output_box = widgets.HBox([table_select_w, field_select_w, interactive_display_output_w])
display(selectors_and_output_box)